In [12]:
import os
import json
import re
from datetime import datetime

from langchain_openai import AzureOpenAIEmbeddings
from openai import OpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv
from groq import Groq
from anthropic import Anthropic

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

# openai_client = AzureOpenAI(
#     api_key=os.getenv("AZURE_CHAT_OPENAI_API_KEY"), 
#     api_version=os.getenv("AZURE_CHAT_OPENAI_API_VERSION"), 
#     azure_endpoint=os.getenv("AZURE_CHAT_OPENAI_ENDPOINT")
#     )
openai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
    )
groq_client = Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY")
    )



embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

In [13]:
log_file_name = ""


def ensure_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)


def log(message):
    global log_file_name
    ensure_directory("logs")
    with open(log_file_name, "a") as file:
        file.write(f"{datetime.now()}\n{message}\n")

In [14]:
conversation_price = 0
conversation_ct = 0
conversation_pt = 0
conversation_tt = 0


def get_conversation_price(new_completion, model="gpt4o"):
    global conversation_price, conversation_ct, conversation_pt, conversation_tt
    USD_price_in_COP = 4100

    if model == "gpt4o-mini":
        in_price = 0.0000006
        out_price = 0.00000015
    if model == "gpt4o":
        in_price = 0.000015
        out_price = 0.000005
    if model == "llama-3-8b":
        in_price = 0.00000005
        out_price = 0.00000008
    if model == "llama-3-70b":
        in_price = 0.00000059
        out_price = 0.0000007
    if model == "haiku":
        in_price = 0.00000025
        out_price = 0.00000125

    ct = new_completion.usage.completion_tokens
    pt = new_completion.usage.prompt_tokens
    tt = new_completion.usage.total_tokens
    usd_total_price = (ct * out_price) + (pt * in_price)
    cop_total_price = usd_total_price * USD_price_in_COP
    conversation_price += cop_total_price
    # conversation_ct += ct
    # conversation_pt += pt
    # conversation_tt += tt
    # conversation_usd_total_price = (conversation_ct * out_price) + (
    #     conversation_pt * in_price
    # )
    # conversation_cop_total_price = conversation_usd_total_price * USD_price_in_COP

    print(
        f"💰 Conversation price: ${conversation_price} COP\n"
        f"   This message: ${cop_total_price} COP "
        f""
    )
    log(
        f"💰 Conversation price: ${conversation_price} COP\n"
        f"   This message: ${cop_total_price} COP "
        f""
    )

    return

In [15]:
### 🔑 Definir system prompt y tools

messages = []
conversation = []
user_input = ""

codes_to_laws = """
- Código Penal: Ley 599 de 2000
- Código Civil: Ley 57 de 1887
- Código de Comercio: Decreto 410 de 1971
- Código de Procedimiento Civil: Decreto 1400 de 1970
- Código de Procedimiento Penal: Ley 906 de 2004
- Código General del Proceso: Ley 1564 de 2012
- Código de la Infancia y la Adolescencia: Ley 1098 de 2006
- Código Nacional de Policía: Decreto 1355 de 1970
- Código de Recursos Naturales: Decreto 2811 de 1974
- Código Electoral: Decreto 2241 de 1986
- Código Disciplinario Único: Ley 734 de 2002
- Código Contencioso Administrativo: Ley 1437 de 2011 (anteriormente Decreto 1 de 1984)
- Código de Minas: Ley 685 de 2001
- Código de Educación: Ley 115 de 1994
- Código Nacional de Tránsito Terrestre: Ley 769 de 2002
- Código del Menor: Decreto 2737 de 1989
- Código de Construcción del Distrito Capital de Bogotá: Acuerdo 20 de 1995
- Código de Construcción Sismo-Resistente: Acuerdo correspondiente (no especificado en los resultados)
- Código de Régimen Departamental: Decreto 1222 de 1986
- Código de Régimen Político y Municipal: Decreto-Ley 1333 de 1986
- Código Penal Militar: Ley 522 de 1999
- Código Penitenciario y Carcelario: Ley 65 de 1993
- Código Procesal del Trabajo y del Seguro Social: Decreto-Ley 2158 de 1948
- Código Sustantivo del Trabajo: Decreto 2663 de 1950
- Código Sanitario Nacional: Ley 9 de 1979
- Código General Disciplinario: Ley 1952 de 2019
- Código Nacional de Seguridad y Convivencia Ciudadana: Ley 1801 de 2016
- CODIGO DE POLICIA DE BOGOTA: ACUERDO 79 DE 2003
- Código Penal Militar: Ley 522 de 1999
- Código Penitenciario y Carcelario: Ley 65 de 1993
- Código Procesal del Trabajo y del Seguro Social: Decreto-Ley 2158 de 1948
- Código Sustantivo del Trabajo: Decreto 2663 de 1950
- Código Sanitario Nacional: Ley 9 de 1979
- Código General Disciplinario: Ley 1952 de 2019
- Código Nacional de Seguridad y Convivencia Ciudadana: Ley 1801 de 2016"""


get_next_chunk = {
    "type": "function",
    "function": {
        "name": "get_next_chunk",
        "description": "Use this tool to get when an important sources comes incomplete or the content is cut off. This tool will help you retrieve the following section of that given source.",
        "parameters": {
            "type": "object",
            "properties": {
                "source_id": {
                    "type": "string",
                    "description": "the unique identifier of the chunk that is incomplete. eg:'20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30'",
                },
            },
            "required": ["source_id"],
        },
    },
}

response_system_template = f"""
Eres Ariel, un asistente de investigación legal en jurisprudencia colombiana. Sigue estas instrucciones al responder:

1. Consulta de Fuentes:
- Tienes acceso a algunas fuentes que previamente has encontrado y seleccionado para generar esta respuesta. - Dentro de esas fuentes se encuentra toda la información disponible para responder. 
- Expresa toda la información que encuentres en las fuentes proporcionadas y que sea relevante para responder.
- Si las fuentes no ayudan a responder la pregunta, responde: "No encuentro información con esos términos, ¿puedes reformular tu consulta?"

2. Redacción de Respuestas:
- Sé detallado y preciso, utilizando exclusivamente la información de las fuentes proporcionadas.
- No incluyas información que no haya sido extraída directamente de las fuentes. Está prohibido.
- No inventes información por ningún motivo. No alucines.

3. Citación de Fuentes:
- Cita el mayor número de fuentes aplicables.
- Utiliza corchetes para referenciar la fuente con el ID del fragmento sin modificarlo, por ejemplo: [12345_id_de_ejemplo].
- No combines fuentes; lista cada fuente por separado.

3. Formato de Respuesta:
- Escribe tus respuestas en formato HTML, sin incluir los tags "```html" al principio o al final.
- Utiliza etiquetas HTML estándar como <p>, <strong>, <em>, etc.

5. Interpretación de Categorías:
- Prioriza las fuentes según su categoría:
    1. Constitucional: Es la norma máxima y más importante establecida por el gobierno del país. 
    2. Legal: Es la que le sigue en importancia y son las leyes dictadas por el Congreso. 
    3. Infralegal: Le sigue en importancia a la categoría Legal y comprende decretos, resoluciones y otros documentos oficiales de autoridades públicas como ministerios y superintendencias. 
    4. Jurisprudencia: Están por debajo de las leyes y las normas infralegales. Son proferidas por jueces de la república y demuestran cómo se ha ejercido la ley en el pasado. Las sentencias y autos de la Corte Constitucional están por encima de la ley,
    5. Doctrina: Son documentos o libros escritos por autores del derecho que no son parte de la ley pero ayudan a darle una interpetación a la ley.
- Da mayor importancia a las fuentes más actuales.

6. Precisión Legal:
- Siempre dale más importancia a la fuente primaria. Si debes citar una ley, artículo o norma, cita la fuente directa, es decir, esa ley, artículo o normal.
- Las fuentes secundarias te ayudarán a entender la interpretación sobre las fuentes primarias.
- No confundas leyes o artículos; si el usuario pregunta por una ley específica, ignora extractos de otras leyes.
- Utiliza las fuentes para dar una respuesta completa y darle al usuario información adicional que pueda serle de ayuda.
- Cita textualmente cuando necesites mencionar el contenido un artículo, ley u otro documento y si el tamaño de este lo permite.

7. Evita Imprecisiones:
- En caso de encontrar información contradictoria, señálala y sugiere una posible causa.
- No te salgas del tema de la pregunta del usuario.
- En caso que el usuario haya dado información incorrecto o imprecisa, señálala y da un argumento.
- Si la ley, la jurisprudencia y la doctrina tienen respuestas diferentes o interpretaciones diferentes para la misma pregunta, señala las distintas perspectivas.
"""

# Incluir que el usuario puede estar proporcionando alguna información que es falsa, en ese caso debe apuntarla y argumentar con la información de las fuentes el porqué es falsa.
# Incluir que señale información contradictoria si es relevante en las fuentes.
# Que siempre le dé más importancia a las fuentes primarias.
# Que las fuentes secundarias ayudan a dar interpretación y contexto.

query_system_template = f"""Eres un asistente de investigación con el objetivo de buscar en una base de conocimiento legal con cientos de miles de documentos, fuentes confiables y precisas acerca de la pregunta del usuario. La base de conocimientos está alojada en Azure AI Search y debes generar un query por cada búsqueda que necesitas y que que conserve todo el significado semántico de la idea. 

Los documentos están guardados en fragmentos con los siguientes campos:
- id: Identificador único del fragmento
- title: Título del documento
- author: Autor del documento
- keywords: Tema legal, puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros.
- category: Tipo de documento legal. Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralegal y Otros_temas_legales. 
- page: Página de la que fue extraído el fragmento.
- year: Año en el que se publicó el documento.
- content: 500 tokens de contenido, fragmento o extracto del documento. Son solo 500 tokens del documento completo.

Pasos para generar la búsqueda:
1. Identifica qué información necesitas buscar para responder de forma precisa y completa al usuario.
2. Solo puedes hacer 3 búsquedas, así que elabora un plan de investigación donde organizarás cómo puedes distribuir tus búsquedas en caso que necesites hacer más de 1.  
3. Genera un query por cada búsquedad que necesitas ejecutar. Puedes generar un máximo de 3 búsquedas. Puedes buscar cualquier cosa, desde teorías del derecho, hasta leyes o una búsqueda general.
4. Cada búsqueda solo puede contener 1 idea. Eso significa que si debes comparar teorías, definiciones, conceptos, leyes, artículos o documentos, debes buscarlos por separado.
5. Por cada búsqueda, da una breve explicación donde argumentes por qué necesitas hacer esa búsqueda.

Cada query debe cumplir con las siguientes checklist:
[] Debe ser una idea semánticamente completa.
[] Debe contener palabras clave. Por ejemplo: conceptos, teorías, artículos, leyes, normas, títulos de documento, o palabras contextuales como 'cuándo', 'dónde', 'lugar', 'momento', 'tiempo', etc.
[] Debe ser conciso y preciso. No debe ser demasiado largo. 

Recomendaciones adicionales:
- No todos los query deben ser específicos para un documento, recuerda que solo puedes hacer máximo 3 búsquedas, así que en ocasiones puedes obtener mejor información si buscas conceptos más generales en vez de un documento específico.
- El query puede ser a forma de pregunta.

Ejemplos de buenos query:
1. Pregunta: '¿Cuándo se entiende por terminada una condena?' - Query: '¿cuándo termina una condena?'
2. Pregunta: '¿Qué dice el artículo 223 del código penal?' - Query: 'artículo 223 Código Penal Ley 599 de 2000 '
3. Pregunta: '¿En qué artículo de la Constitución se consagra el bloque de constitucionalidad?' - Query: 'bloque de constitucionalidad Constitución'
4. Pregunta: '¿Cuándo se consuma el hurto?' - Query: '¿cuándo se consuma el hurto?'

Esta es una lista de los códigos en colombia y sus respectiva ley. Si necesitas buscar alguna de estas fuentes, debes incluir ambas refencias en tu query: {codes_to_laws}

Genera tu respuesta en formato JSON Array. Aunque generes solo 1 búsqueda, mete el JSON en un array[] con el siguiente schema:
{{"searches":[{{"search_query":"El query de búsqueda completo. Este debe cumplir con todos los requitiso.","reason":"Argumento de porqué necesitas hacer esta búsqueda."}}]}}
"""


query_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "search_query",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "searches": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "search_query": {
                                "type": "string",
                                "description": "El query de búsqueda completo. Este debe cumplir con todos los requitiso.",
                            },
                            "reason": {
                                "type": "string",
                                "description": "Argumento de porqué necesitas hacer esta búsqueda.",
                            },
                        },
                        "required": [
                            "search_query",
                            "reason",
                        ],
                        "additionalProperties": False,
                    },
                },
                #     "strategy": {
                #         "type": "string",
                #         "description": "Describe brevemente qué estrategia de investigación vas a ejecutar y porqué.",
                #     },
            },
            "required": ["searches"],
            # "required": ["searches", "strategy"],
            "additionalProperties": False,
        },
    },
}
selected_sources_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "selected_sources",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "sources": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "source_id": {
                                "type": "string",
                                "description": "ID de la fuente que seleccionaste",
                            },
                            "reason": {
                                "type": "string",
                                "description": "Argumento de porqué seleccionaste esta fuente.",
                            },
                        },
                        "required": [
                            "source_id",
                            "reason",
                        ],
                        "additionalProperties": False,
                    },
                },
                "selection_strategy": {
                    "type": "string",
                    "description": "Describe brevemente porqué seleccionaste de esta manera las fuentes. ¿Cuál fue tu estrategia y porqué no seleccionaste otros documentos?",
                },
            },
            "required": ["sources", "selection_strategy"],
            "additionalProperties": False,
        },
    },
}
# Agregar un campo para identificar fuentes cortadas

In [16]:
def generate_completion(completion_messages):
    log(f"{json.dumps(completion_messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=completion_messages,
        tools=[get_next_chunk],
        temperature=0.2,
        n=1,
        tool_choice="auto",
    )
    log(response)
    get_conversation_price(response)
    return response


def choose_sources_completion(completion_messages):
    log(f"{json.dumps(completion_messages, indent=4)}\n")
    response = groq_client.messages.create(
        max_tokens=2000,
        model="claude-3-haiku-20240307",
        messages=completion_messages,
        temperature=0.0,
    )
    log(response)
    get_conversation_price(response, model="haiku")
    return response


def generate_query(query_messages):
    log(f"{json.dumps(query_messages, indent=4)}\n")
    response = groq_client.messages.create(
        max_tokens=2000,
        model="claude-3-haiku-20240307",
        messages=query_messages,
        temperature=0.0,
    )
    log(response)
    get_conversation_price(response, model="haiku")
    return response

In [17]:
def format_search_results(docs_list):
    print(f"📋 Agarrando todos los resultados ...")
    documents = []
    try:
        docs_list = list(docs_list)
        print(f"📋 Formateando resultados ...")
        for index, document in enumerate(docs_list, start=1):
            captions: QueryCaptionResult = document.get("@search.captions", "")
            captions_text = " // ".join([caption.text for caption in captions]) if captions is not None else ""
            doc_formatted = {
                "position" : index,
                "score": document.get("@search.score", 10),
                "rerank": document.get("@search.reranker_score", 10),
                "captions": captions_text,
                "id": document["id"],
                "title": document["title"],
                "author": document["author"],
                "keywords": document["keywords"],
                "category": document["category"],
                "page": document["page"],
                "year": document["year"],
                "has_copyright": document["has_copyright"],
                "file_path": document["file_path"],
                "external_id": document["external_id"],
                "content": document["content"],
            }
            documents.append(doc_formatted)
    except Exception as e:
        print(e)
    log(
        f">>>>>>format_search_results:\n"
        f"{json.dumps(documents, indent=4)}"
        )
    return documents

def filter_docs(doc_list):
    print(f"🧹 Eliminando fuentes irrelevantes o duplicadas ...")
    filtered_docs = []
    seen_docs = set()
    
    for doc in doc_list:
        score = doc.get("score", 10) 
        rerank = doc.get("rerank", 10) 
        content = doc["content"] 
        if (
            content not in seen_docs
            and (rerank > 2
            or score > 0.17)
        ):
            filtered_docs.append(doc)
            seen_docs.add(content)
    print(f"🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...")
    log(f"🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...\n"
    f"{json.dumps(filtered_docs, indent=4)}")
    return filtered_docs

def add_doc_to_context(docs_list):
    print(f"📋 Agregando fuentes al contexto ...")
    sources = ""
    counter = 0
    
    for document in docs_list:
        counter += 1
        sources += (
            f"Fuente:\n"
            f"{document.get("reason", "")}"
            f"id: {document["id"]}\n"
            f"title: {document["title"]}\n"
            f"author: {document["author"]}\n"
            f"year: {document["year"]}\n"
            f"keywords: {document["keywords"]}\n"
            f"category: {document["category"]}\n"
            f"page: {document["page"]}\n"
            f"content: {document["content"]}\n"
            f"\n\n"
        )      
    log(
        f"To add in Context: \n"
        f"{json.dumps({"sources":sources}, indent=4)}"
        )
    print(f"📋 {counter} fuentes agregadas ...")
    return sources
    

In [18]:
def search_for_chunks(text_query):
    log(f"🔎 Buscando documentos: {text_query} ...")
    print(f"🔎 Buscando documentos: {text_query} ...")
    results_num = 50
    vector_query = embeddings_client.generate_embeddings(content=text_query)

    results = search_client.search(
        search_text=text_query,
        vector_queries=[
            {
                "vector": vector_query,
                "k": results_num,
                "fields": "content_vector",
                "kind": "vector",
                # "exhaustive": True,
            }
        ],
        top=results_num,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )
    results_formatted = format_search_results(results)
    restults_filtered = filter_docs(results_formatted)

    return restults_filtered[:25]


def get_next_chunks(id):
    log(f"⏩️ Buscando los chunks siguientes de {id} ...")
    print(f"⏩️ Buscando los chunks siguientes de {id} ...")
    pattern = r"^(.*_chunk)(\d+)$"
    match = re.search(pattern, id)

    if match:
        prefix = match.group(1)
        chunk_number = int(match.group(2))
        next_chunks_result = search_client.search(
            filter=f"id eq '{prefix}{chunk_number + 1}' or id eq '{prefix}{chunk_number + 2}'",
            top=2,
        )
        return format_search_results(list(next_chunks_result))


def get_next_chunk_tool(source_id):
    next_chunks = get_next_chunks(source_id)
    log(f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}")
    return f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}"

In [19]:
def append_message(new_message):
    global messages, conversation

    if isinstance(new_message, dict):
        role = new_message.get("role")
        content = new_message.get("content")
    else:
        role = new_message.role
        content = new_message.content

    if role == "assistant":
        if new_message.tool_calls:
            tool_calls = new_message.tool_calls
            tool_calls_formatted = []
            for tool_call in tool_calls:
                tool_calls_formatted.append(
                    {
                        "id": tool_call.id,
                        "function": {
                            "arguments": str(json.loads(tool_call.function.arguments)),
                            "name": tool_call.function.name,
                        },
                        "type": "function",
                    }
                )
            messages.append(
                {
                    "role": "assistant",
                    "tool_calls": tool_calls_formatted,
                },
            )
            return messages  ## Assistant call tools
        elif content:
            messages.append({"role": "assistant", "content": content})
            return messages  ## Assistant talk
    elif role == "tool":
        messages.append(new_message)
        return messages  ## Tool reponse
    else:
        return messages

In [20]:
def call_tools(tool_calls):
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        if tool_name == "get_next_chunk":
            content = get_next_chunk_tool(tool_args.get("source_id"))

        new_tool_message = {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": tool_name,
            "content": content,
        }

        append_message(new_message=new_tool_message)

    print(">> 🤖 Generando respuesta ...")
    new_assistant_completion = generate_completion()
    new_assistant_message = new_assistant_completion.choices[0].message
    append_message(new_message=new_assistant_message)

    if new_assistant_message.content:
        return print("💬 Assistant:" + new_assistant_message.content)
    else:
        return call_tools(
            tool_calls=new_assistant_message.tool_calls,
        )

In [21]:
def run_conversation(user_prompt=None):
    if user_prompt:
        get_answer(user_prompt)
    else:
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Exiting chat...")
                break
            get_answer(user_input)


def get_answer(user_prompt):
    global messages, log_file_name, user_input

    user_input = user_prompt

    current_time = datetime.now().strftime("%m-%d %H:%M")
    log_file_name = f"logs/{current_time} - {user_input[:50]}.log"

    print("🙎‍♂️ User: " + user_input)
    log(f"User input: {user_input}")

    query_prompt_message = {"role": "system", "content": query_system_template}
    response_prompt_message = {"role": "system", "content": response_system_template}
    query_user_prompt_message = {
        "role": "user",
        "content": f"Genera uno o varias búsquedas para responder esta pregunta: {user_input}",
    }

    query_messages = [
        query_prompt_message,
        *messages,
        query_user_prompt_message,
    ]

    print("🤖 Generando queries con la pregunta del usuario ...")
    first_query_completion = generate_query(query_messages)
    first_search_terms = json.loads(
        first_query_completion.choices[0].message.content
    ).get("searches")
    print(f"🤖 {len(first_search_terms)} búsquedas generadas ...")

    sources_found = []
    search_result_prompt = ""
    search_reasons = ""

    for search in first_search_terms:
        search_query = search.get("search_query")
        query_reason = search.get("reason")
        # print(f"Buscando '{search_query}': {query_reason}")
        print(query_reason)
        query_results = search_for_chunks(search_query)
        sources_found += query_results
        search_reasons += f"{query_reason} Por eso buscaré '{search_query}'.\n"
        # search_queries += f"'{search_query}',"
        sources_to_add = add_doc_to_context(query_results)
        search_result_prompt += f"{query_reason}.\nPor eso búsqué '{search_query}' y encontré estas fuentes:\n{sources_to_add}\n\n"

    # Meter aquí razonamiento de qué información ha encontrado en esta búsqueda.

    reasoning_results_prompt_message = {
        "role": "assistant",
        "content": search_result_prompt,
    }
    reasoning_prompt_message = {
        "role": "system",
        "content": f"Necesito que plantees una nueva y mejorada estrategia de búsqueda para responder al usuario de forma completa:\n1. Resume La Información: Lee y absorbe toda la información de las fuentes y utiliza los hallazgos más relevantes para mi investigación.\n2. Genera Una Nueva Búsqueda: Con estos nuevos hallazgos sugiere un máximo de 3 búsquedas que ayuden a conseguir información precisa y relevante para reponder a la pregunta del usuario.\n3. Dime por qué estas 3 nuevas búsquedas me ayudarían en mi investigación.\nRecuerda que cada query debe cumplir con el checklist y que estamos buscando documentos para responder a: {user_input}",
    }

    query_messages += [
        reasoning_results_prompt_message,
        reasoning_prompt_message,
    ]

    print("🤖 Generando nuevas búsquedas con la nueva información ...")
    second_query_completion = generate_query(query_messages)
    second_search_terms = json.loads(
        second_query_completion.choices[0].message.content
    ).get("searches")
    print(f"🤖 {len(second_search_terms)} búsquedas generadas ...")

    query_reasons = ""
    # search_strategy = first_search_terms.get("strategy")
    # print(f"{search_strategy}")
    for search in second_search_terms:
        search_query = search.get("search_query")
        query_reason = search.get("reason", "")
        print(f"Buscando '{search_query}': {query_reason}")
        # print(query_reason)
        query_results = search_for_chunks(search_query)
        sources_found += query_results
        query_reasons += f"{query_reason} Por eso busqué '{search_query}'. "

    # return
    sources_found_filtered = filter_docs(sources_found)
    sources_to_add = add_doc_to_context(sources_found_filtered)

    response_prompt_message = {"role": "system", "content": f"""{response_system_template}\n\nGenera tu respuesta en formato JSON con el siguiente schema:{{"sources":[{{"source_id":"ID de la fuente que seleccionaste"}}]}}"""}
    select_user_prompt_message = {
        "role": "user",
        "content": f"Busca información relevante y selecciona las mejores fuentes para responder a: {user_input}",
    }
    select_prompt_message = {
        "role": "assistant",
        "content": f"""{query_reasons}.\n\nResultados de mis búsquedas:\n{sources_to_add}\n\nEjectuaste una búsqueda con lo que el usuario te preguntó y encontraste fuentes que pueden servirte, ahora debes escoger las mejores. Sigue las siguientes instrucciones:\n1. Revisa todos los extractos obtenidos con tu búsqueda y selecciona mínimo 3 y máximo 10 fuentes que te sirvan para responder al usuario. No pueden ser menos de 3 y al menos 2 de esas deben ser Jurisprudencia o Doctrina.\n2. Enlista las fuentes que escogiste e identifícalas por su ID y por ningún motivo en absoluto cambies el valor del ID o número de chunk de una fuente. Ordénalas por importancia, posicionando de primero la que consideres más relevante o más cercanas a lo que el usuario pidió.\n3.(Opcional) Intenta seleccionar un grupo de fuentes donde puedas dar una respuesta completa a la pregunta, señalar contradicciones en las fuentes, dar información complementaria relevante y señalar diversos puntos de vista al respecto lo que el usuario pregunta.\n4. Al lado de cada fuente que seleccionaste escribe una muy breve explicación argumentando porqué la escogiste y porqué ayuda a responder mejor la solicitud del usuario.\n5. Toda la información debe provenir de lo que dicen las fuentes, no inventes ni des información adicional que no estén presentes en las fuentes.\n6. Debes seleccionar al menos 2 fuentes que sean Jurisprudencia o Doctrina, ya que estos ayudarán a dar contexto a las leyes en su aplicación.\nRecomendaciones adicionales\n- Dale más importancia a las fuentes que son extractos del documento que el usuario pidio.\n- Conforma tu selección tanto de fuentes primarias como secundarias si esto ayuda a reponder mejor la pregunta.\n- Las fuentes que selecciones es la única información que vas a tener disponible para responder a la pregunta, así que debes escoger fuentes que dentro de su contenido se encuentre la información.\n- En tu razonamiento, si escogiste una fuente porque incluye cierta ley, artículo, teoría o mención de algo especial, qué ley, artículo, teoría, etc es a la que te refieres.\n- No selecciones fuentes repetitivas, busca tener diversidad en tus fuentes.\n- En el ID de la fuente encontrarás el número del chunk ej: '..._chunk10', esto significa que el siguiente chunk ('...chunk11') es el fragmento que le sigue al documento. Ten en cuenta esto para que sepas que si un contenido importante está cortado en un chunk, puedes agarrar el siguientes también.\n- Ten en cuenta que los nombres de algunos codigos hacen referencia a una ley, decreto o articulo. Aqui tienes un listado para que te guies:\n{codes_to_laws}""",
    }

    selection_messages = [
        response_prompt_message,
        *messages,
        select_user_prompt_message,
        select_prompt_message,
        # select_instructions_prompt_message,
    ]

    print("🤖 Escogiendo las mejores fuentes para responder al usuario ...")
    selected_sources_to_add = []
    sources_at_end = ""
    selected_sources_completion = choose_sources_completion(selection_messages)
    selected_sources = json.loads(
        selected_sources_completion.choices[0].message.content
    ).get("sources")
    for index, source in enumerate(selected_sources, start=1):
        source_id = source.get("source_id")
        source_reason = source.get("reason")
        for source in sources_found_filtered:
            if source["id"] == source_id:
                source["reason"] = source_reason
                sources_at_end += f"{index}. {source["title"]} - Página {source["page"]}<br>\n"
                selected_sources_to_add.append(source)
        print(f"{index}.[{source_id}]: {source_reason}")

    # return
    sources_to_add = add_doc_to_context(selected_sources_to_add)
    # TODO: Agregar hasta 3 por cada query que excedan 2.8/3 como score semántico.
    # Quizás ponerle cap de 6 fuentes máximo en total ordenadas por score semántico, porque pueden ser 18 si ambas búsqueda es de 3 queries y tiene matchs fuertes
    # TODO: Esto podría ser que los reorganice convirtiendo en vectores en la pregunta y comparándolos con los vectores de cada chunk, ChatGPT me dió esa idea... o cohere.

    user_message = {
        "role": "user",
        "content": user_input,
    }
    sources_prompt_message = {
        "role": "assistant",
        "content": f"Fuentes econtradas:\n{sources_to_add}",
    }

    response_messages = [
        response_prompt_message,
        *messages,
        user_message,
        sources_prompt_message,
    ]

    response_completion = generate_completion(response_messages)
    print(response_completion.choices[0].message.content)
    
    
    
    print(f"\n\n<p>{sources_at_end}</p>")

Probar con all > Dejarlo default mejor ✅
Probar full text search > Malos resultados ✅
Filtrar por semantic rerank score de 2 o más. ✅
Filtrar por score de 1.7 o más. ✅
Incluir hasta 25 fuentes por query.
Otro proceso del mini 4o donde seleccione las mejores fuentes para responder la respuesta ✅
Pedir que razone porqué escogió esos query ✅
Probar corriendo el proyecto en un .py normal y ponerle timers.


In [22]:
run_conversation("¿Qué dice el artículo 250 de la Constitución?")

🙎‍♂️ User: ¿Qué dice el artículo 250 de la Constitución?
🤖 Generando queries con la pregunta del usuario ...


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}

In [108]:
run_conversation(
    "Tienes el siguiente caso: Un estudiante de sexto de bachillerato de un colegio privado católico es echado del colegio por tener el pelo largo. Él laega que en el libre desarrollo de su personalidad, puede llevar el pelo como quiera. Por eso, presentó una tutela para que lo reintegren al colegio. El colegio ha contestado a la tutela indicando que, al ser una institución privada, ellos pueden reservarse el derecho de admisión y dictarse su propio manual de convivencia, en el que está prohibido llevar el pelo largo. Tú eres el juez de la tutela: ¿en qué sentido emites tu decisión?"
)

🙎‍♂️ User: Tienes el siguiente caso: Un estudiante de sexto de bachillerato de un colegio privado católico es echado del colegio por tener el pelo largo. Él laega que en el libre desarrollo de su personalidad, puede llevar el pelo como quiera. Por eso, presentó una tutela para que lo reintegren al colegio. El colegio ha contestado a la tutela indicando que, al ser una institución privada, ellos pueden reservarse el derecho de admisión y dictarse su propio manual de convivencia, en el que está prohibido llevar el pelo largo. Tú eres el juez de la tutela: ¿en qué sentido emites tu decisión?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $436.248323 COP
   This message: $4.927544 COP 
🤖 3 búsquedas generadas ...
Necesito buscar la definición y alcance del libre desarrollo de la personalidad en la Constitución de Colombia para entender los derechos del estudiante.
🔎 Buscando documentos: libre desarrollo de la personalidad en la Constitución de Colombia ...
📋 Agar

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j85f9s7yfkrvaszq50j89nn0` on tokens per minute (TPM): Limit 20000, Used 0, Requested 40808. Please try again in 1m2.424s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [43]:
run_conversation(
    "¿Cuáles son las principales sentencias de la Corte Constitucional sobre el aborto?"
)

🙎‍♂️ User: ¿Cuáles son las principales sentencias de la Corte Constitucional sobre el aborto?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $278.58392999999995 COP (In: 428534, Out: 6112, Total token: 434646)
   This message: $1.2386100000000002 COP (In: 1442, Out: 143, Total token: 1585) 
🤖 3 búsquedas generadas ...
Necesito buscar las sentencias más relevantes de la Corte Constitucional sobre el aborto para proporcionar un resumen de las decisiones y su impacto en la legislación y derechos en Colombia.
🔎 Buscando documentos: principales sentencias Corte Constitucional aborto ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 50 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
Es importante encontrar documentos que contengan jurisprudencia específica de la Corte Constitucional relacionada con el aborto, para entender el contexto legal y la

In [44]:
run_conversation("¿Qué dice el artículo 103 del código penal?")

🙎‍♂️ User: ¿En qué casos la jurisprudencia ha reconocido la dosis de aprovisionamiento?
🤖 Generando queries con la pregunta del usuario ...
💰 Conversation price: $339.67434 COP (In: 521468, Out: 7712, Total token: 529180)
   This message: $1.28781 COP (In: 1442, Out: 163, Total token: 1605) 
🤖 3 búsquedas generadas ...
Necesito buscar información sobre la jurisprudencia que ha reconocido la dosis de aprovisionamiento, para entender en qué casos se ha aplicado y cuáles son los criterios utilizados por los tribunales.
🔎 Buscando documentos: dosis de aprovisionamiento jurisprudencia ...
📋 Agarrando todos los resultados ...
📋 Formateando resultados ...
🧹 Eliminando fuentes irrelevantes o duplicadas ...
🧹 Docs filtrados. De 50 pasaron 32 ...
📋 Agregando fuentes al contexto ...
📋 25 fuentes agregadas ...
Es importante identificar casos específicos en los que se ha discutido la dosis de aprovisionamiento en la jurisprudencia, para tener ejemplos concretos y entender su aplicación práctica.
🔎 

In [ ]:
run_conversation("¿Cuándo se consuma el hurto?")